In [1]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import MNIST, STL10
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
import tarfile
from torch import nn
import shutil
from tqdm import tqdm
from advbench.scripts.train import PD_ALGORITHMS

In [2]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn

import torch
import random
import numpy as np
import os
import json
import pandas as pd
import time
from humanfriendly import format_timespan


from advbench import datasets
from advbench import algorithms
from advbench import attacks
from advbench import hparams_registry
from advbench.lib import misc, meters, plotting, logging
from torch.cuda.amp import autocast
from torchsummary import summary
DEVICE = "cuda:1"
DOWNLOAD_WEIGHTS = True
PULL_METRICS = True

In [3]:
def load_weights(weight_path, dataset, device=DEVICE, hparams={}):       
        kw_args = {"perturbation": hparams["perturbation"]}
        if hparams["algorithm"] in PD_ALGORITHMS: 
            if hparams["algorithm"].endswith("Reverse"):
                kw_args["init"] = 0.0
            else:
                kw_args["init"] = 1.0
        print(hparams["perturbation"])
        algorithm = vars(algorithms)[hparams["algorithm"]](
        dataset.INPUT_SHAPE, 
        dataset.NUM_CLASSES,
        hparams,
        device,
        **kw_args).to(device)
        w_dict = torch.load(weight_path,map_location=DEVICE)
        #print(weight_dict.keys())
        #if db=="mnist":
        weight_dict = {}
        for s,v in w_dict.items():
                if "classifier" in s and "model" not in s:
                        weight_dict[s.replace('classifier', 'classifier.model')] = v
                else:
                        weight_dict[s] = v 
        algorithm.load_state_dict(weight_dict)
        return algorithm

In [ ]:
if PULL_METRICS:
    api = wandb.Api(timeout=50)
    dsets = ["modelnet40"]#["STL10", "CIFAR100"]
    #results_dfs = {}
    entity = "hounie"
    all_runs_list = []
    for dataset in dsets:
        print("Fetching data for {}".format(dataset))
        project = f"OOD-{dataset}"
        base = f"../trained_weights/{project}"
        Path( base ).mkdir( parents=True, exist_ok=True )
        runs = api.runs(f"{entity}/{project}")
        print(f"{len(runs)} runs found")
        for run in tqdm(runs):
            if run.state == "finished" or (dataset=="modelnet40" and run.state != "running"):
                id = run.id
                run = api.run(f"{entity}/{project}/{id}")
                tmp_path = os.path.join(base, f"tmp")
                weight_dir = os.path.join(base, run.name)
                path = os.path.join(weight_dir, f"{run.id}.pkl")
                Path( weight_dir ).mkdir( parents=True, exist_ok=True )
                results = {**run.summary, **run.config, "weights": path, "id": run.id, "name":run.name}
                if DOWNLOAD_WEIGHTS:
                    try:
                        print(f"train-output/{run.name[:-4]} DGCNN {run.name[-1]}_ckpt.pkl")
                        f = run.file(f"train-output/{run.name[:-4]} DGCNN {run.name[-1]}_ckpt.pkl").download(tmp_path, replace=True) 
                        os.rename(f.name, path)
                        all_runs_list.append(results)
                    except:
                        print(f"{run.name} {run.id} failed")
                        
    df = pd.DataFrame(all_runs_list)
    df.to_csv(f"./results.csv")

Fetching data for modelnet40
72 runs found


  0%|                                                                            | 0/72 [00:00<?, ?it/s]

train-output/finalPointcloudJitter Adversarial_Smoothed DGCNN 4_ckpt.pkl


  3%|█▉                                                                  | 2/72 [00:00<00:11,  6.11it/s]

finalPointcloudJitter Adversarial_Smoothed 0 4 29ebt6zd failed
train-output/paretoPointcloudJitter Beta_PD_Reverse DGCNN 0_ckpt.pkl


  4%|██▊                                                                 | 3/72 [00:01<00:28,  2.41it/s]

train-output/paretoPointcloudJitter Beta_PD_Reverse DGCNN 0_ckpt.pkl


In [ ]:
df = pd.read_csv("./results.csv")
test_keys = []
train_keys = []
for c in df.columns:
        if "loss" not in c and "acc" not in c:
            if c.startswith("test"):
                test_keys.append(c)
            else:
                train_keys.append(c)
    

In [ ]:
from tqdm import tqdm
CLEAN_METRICS = False
data_dir = '../advbench/data'
device = DEVICE
failed = []
n_aug = {"Fo_PGD": 1}
t_attacks = [k for k in n_aug.keys()]
for exp_id in tqdm(df["id"]):
        if True:
            api = wandb.Api(timeout=50)
            exp = df[df["id"]==exp_id]
            dataset =  exp["dataset"].values[0]
            perturbation = exp["perturbation"].values[0]
            print(perturbation)
            model = exp["model"].values[0]
            if True:
                project = f"OOD-{dataset}"
                t_hparams = exp[test_keys].to_dict(orient='index')[exp.index.values[0]]
                test_hparams = {}
                for k, v in t_hparams.items():
                    test_hparams[k.replace("test_","")] = v
                print(exp.perturbation.values[0])
                if exp.perturbation.values[0] == 'SE':
                    hparams['epsilon'] = torch.tensor([hparams[f'epsilon_{i}'] for i in ("rot","tx","ty")]).to(device)
                    test_hparams['epsilon'] = torch.tensor([test_hparams[f'epsilon_{tfm}'] for tfm in ("rot","tx","ty")]).to(device)
                elif exp.perturbation.values[0] =='PointcloudTranslation':
                    hparams['epsilon'] = torch.tensor([hparams['epsilon_tx'] for i in range(3)] + [hparams['epsilon_ty'] for i in range(3)]).to(device)
                    test_hparams['epsilon'] = torch.tensor([test_hparams['epsilon_tx'] for i in range(3)] + [test_hparams['epsilon_ty'] for i in range(3)]).to(device)

                hparams = exp[train_keys].to_dict(orient='index')[exp.index.values[0]]
                hparams['model'] = model
                test_hparams['epsilon'] = hparams['epsilon']
                print(hparams['epsilon'])
                hparams['batched'] = True
                test_hparams['batched'] = True
                if "augment" in hparams:
                    aug = hparams["augment"]
                else:
                    aug = True
                dataset = vars(datasets)[dataset](data_dir, augmentation = aug)
                train_ldr, val_ldr, test_ldr = datasets.to_loaders(dataset, hparams, device=device)
                algorithm = load_weights(exp["weights"].values[0], dataset, device=DEVICE, hparams=hparams)           
                test_attacks = {a: vars(attacks)[a](algorithm.classifier, test_hparams, DEVICE, perturbation=hparams["perturbation"]) for a in t_attacks}
                wandb.init(project=project, resume=exp_id)
                if CLEAN_METRICS:
                    train_clean_acc, train_clean_mean_acc, train_clean_loss = misc.accuracy_mean_overall_loss(algorithm, train_ldr, device, max_batches=100)
                    val_clean_acc, val_clean_mean_acc, val_clean_loss = misc.accuracy_mean_overall_loss(algorithm, val_ldr, device)
                    test_clean_acc, test_clean_mean_acc, test_clean_loss = misc.accuracy_mean_overall_loss(algorithm, test_ldr, device)
                    wandb.log({'train_clean_loss': train_clean_loss,'train_clean_acc': train_clean_acc, 'train_clean_acc_bal': train_clean_mean_acc})
                    wandb.log({'best_val_clean_loss': val_clean_loss, 'best_val_clean_acc': val_clean_acc, 'best_val_clean_acc_bal': val_clean_mean_acc})
                    wandb.log({'test_clean_loss': test_clean_loss,'test_clean_acc': test_clean_acc, 'test_clean_acc_bal': test_clean_mean_acc})
                for attack_name, attack in test_attacks.items():
                    print(f"Logging {attack_name}...")
                    test_adv_acc, test_adv_acc_mean, test_adv_acc_bal, test_adv_acc_mean_bal, adv_loss, accs, loss, deltas = misc.adv_accuracy_loss_delta_balanced(algorithm, test_ldr, device, attack, augs_per_batch=1)
                    print("Test Adversarial Accuracy:", test_adv_acc)
                    print("Test Balanced Adversarial Accuracy:", test_adv_acc_bal)
                    train_adv_acc, train_adv_acc_mean, train_adv_acc_bal, train_adv_acc_mean_bal, adv_loss, train_accs, train_loss, train_deltas = misc.adv_accuracy_loss_delta_balanced(algorithm, val_ldr, device, attack, augs_per_batch=1)
                    print("Train Adversarial Accuracy:", train_adv_acc)
                    print("Train Balanced Adversarial Accuracy:", train_adv_acc_bal)
                    wandb.log({'test_acc_adv_'+attack_name: test_adv_acc,'test_acc_adv_mean_'+attack_name: test_adv_acc_mean, 'test_loss_adv_'+attack_name: loss.mean(),
                     'test_acc_adv_bal_'+attack_name: test_adv_acc_bal, 'test_acc_adv_mean_bal_'+attack_name: test_adv_acc_mean_bal})
                    wandb.log({'train_acc_adv_'+attack_name: train_adv_acc,'train_acc_adv_mean_'+attack_name: train_adv_acc_mean, 'train_loss_adv_'+attack_name: loss.mean(),
                     'train_acc_adv_bal_'+attack_name: train_adv_acc_bal, 'train_acc_adv_mean_bal_'+attack_name: train_adv_acc_mean_bal})                                                   
                wandb.finish(quiet=True)
        else:
            failed.append(exp_id)

with open('failed.txt', 'w') as f:
    for line in failed:
        f.write(f"{line}\n")